# Value gradient error for linear policies in LQG

Experiment description on [Overleaf](https://www.overleaf.com/read/cmbgmxxpxqzr).

**Versioning:** [CalVer](https://calver.org) `MM.DD.MICRO`

In [1]:
from __future__ import annotations
from datetime import date

import lqsvg.torch.named as nt
import pytorch_lightning as pl
import torch
import wandb
from raylab.policy.model_based.lightning import LightningTrainerSpec
from torch import Tensor

from data import build_datamodule
from models import LightningModel
from policy import make_worker
from utils import suppress_dataloader_warning

In [2]:
def make_lightning_trainer(logger: pl.loggers.LightningLoggerBase, spec: LightningTrainerSpec) -> pl.Trainer:
    early_stopping = pl.callbacks.EarlyStopping(
        monitor=LightningModel.early_stop_on,
        min_delta=spec.improvement_delta,
        patience=spec.patience,
        mode="min",
        strict=True,
    )
    checkpointing = pl.callbacks.ModelCheckpoint(
        monitor=LightningModel.early_stop_on,
        save_top_k=-1, 
        period=10, 
        save_last=True
    )
    trainer = pl.Trainer(
        logger=logger,
        num_sanity_val_steps=2,
        callbacks=[early_stopping, checkpointing],
        max_epochs=spec.max_epochs,
        max_steps=spec.max_steps,
    )
    return trainer

In [3]:
def calver() -> str:
    today = date.today()
    return f"{today.month}.{today.day}.0"

print("CalVer:", calver())

CalVer: 3.11.0


In [4]:
run = wandb.init(
    name="SVG Prediction",
    project="LQG-SVG",
    entity="angelovtt",
    tags=[calver()],
    reinit=True,
    mode="online",
    save_code=True,
)

with nt.suppress_named_tensor_warning():
    env_config = dict(n_state=2, n_ctrl=2, horizon=100, num_envs=100)
    worker = make_worker(env_config)
    model = LightningModel(worker.get_policy(), worker.env)
    model.hparams.learning_rate = 1e-3
    model.hparams.update(env_config)
    run.config.update(model.hparams)
    datamodule = build_datamodule(worker, total_trajs=5000)

    logger = pl.loggers.WandbLogger(log_model=True, experiment=run)
    spec = LightningTrainerSpec(max_epochs=2000, patience=3, improvement_delta=0.0)
    trainer = make_lightning_trainer(logger, spec)

wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Collecting:   0%|          | 0/5000 [00:00<?, ?traj/s]

2021-03-11 14:30:09,492	WARNING deprecation.py:29 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [12]:
print(pl.__version__)

1.1.5


In [7]:
logger.save_dir

In [4]:
with run, suppress_dataloader_warning():
    trainer.fit(model, datamodule=datamodule)
    results = trainer.test(model, datamodule=datamodule)
    run.log(results[0])

wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Collecting:   0%|          | 0/5000 [00:00<?, ?traj/s]

2021-03-11 13:52:57,658	WARNING deprecation.py:29 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name        | Type           | Params
-----------------------------------------------
0 | actor       | TVLinearPolicy | 600   
1 | model       | LQGModule      | 3.6 K 
2 | mdp         | LQGModule      | 3.6 K 
3 | policy_loss | PolicyLoss     | 0     
-----------------------------------------------
7.8 K     Trainable params
0         Non-trainable params
7.8 K     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Saving latest checkpoint...


train/loss,290.93225
epoch,247
_runtime,520
_timestamp,1615482094
_step,13639
val/loss,292.27533
val/monte_carlo_value,-395.13431
val/monte_carlo_svg_norm,289.46246
val/analytic_value,-401.34195
val/analytic_svg_norm,286.88931
val/monte_carlo_diff,6.15265


train/loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/monte_carlo_value,█▆▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂
val/monte_carlo_svg_norm,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇█▇█████████
val/analytic_value,█▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val/analytic_svg_norm,▁▁▂▂▃▃▄▄▅▅▅▆▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█████████
val/monte_carlo_diff,█▆▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂


TypeError: expected str, bytes or os.PathLike object, not NoneType